In [54]:

import matplotlib.pyplot as plt
import numpy as np
import helper

import torch.nn as nn
import torchvision.models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
import torch
import pandas as pd
from torchinfo import summary
from PIL import Image
from torchvision.transforms import ToTensor
from glob import glob
from torch.utils.data import Dataset, DataLoader, random_split
from copy import copy
from collections import defaultdict
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
from sklearn.metrics import classification_report
from tqdm import tqdm
import math
from torcheval.metrics import BinaryAccuracy
import os
import torchmetrics
import timm
import time
import datetime
import random
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size=4
image_count=50
img_size=256
tf = ToTensor()

In [56]:
class CustomDataset(Dataset):
    def __init__(self, id,image_list, label_list):
        self.img_path = image_list

        self.label = label_list
        self.id=id

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        id_tensor=self.id[idx]
        image_tensor = self.img_path[idx]
    
        label_tensor =  self.label[idx]
        return image_tensor, label_tensor


train_data=pd.read_csv('../../data/mteg_data/train_label.csv') 
file_path='../../data/mteg_data/frame/train/'
train_image_list=[]
for i in range(len(train_data)):
    file_name=train_data.loc[i]['File Name']
    id=file_name[:file_name.find('_')]
    train_image_list.append(file_path+id)
label_data=pd.read_csv('../../data/mteg_data/label.csv')  
train_label_list=[]
train_id_list=[]
train_image_tensor = torch.empty((len(train_image_list),image_count,3, img_size, img_size))
for i in tqdm(range(len(train_image_list))):
    folder_name=os.path.basename(train_image_list[i])
    dst_label=label_data.loc[label_data['일련번호']==int(folder_name[:-1])]
    dst_label=dst_label.loc[dst_label['구분값']==int(folder_name[-1])].reset_index()
    label=int(dst_label.loc[0]['OTE 원인'])
    train_id_list.append(folder_name)
    train_label_list.append(label-1) 
    image_file_list = glob(train_image_list[i]+'/*.jpg')
    if len(image_file_list)>image_count:
        image_index = torch.randint(low=0, high=len(
            image_file_list)-image_count, size=(1,))
        count = 0
        for index in range(image_count):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            train_image_tensor[i,count] = image
            count += 1
    else:
        count = 0
        for index in range(len(image_file_list)):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            train_image_tensor[i,count] = image
            count += 1
        for j in range(image_count-count):
            image = 1-tf(Image.open(image_file_list[j]).resize((img_size,img_size)))
            train_image_tensor[i,count] = image
            count += 1
            
val_data=pd.read_csv('../../data/mteg_data/val_label.csv') 
file_path='../../data/mteg_data/frame/val/'
val_image_list=[]
for i in range(len(val_data)):
    file_name=val_data.loc[i]['File Name']
    id=file_name[:file_name.find('_')]
    val_image_list.append(file_path+id)
label_data=pd.read_csv('../../data/mteg_data/label.csv')  
val_label_list=[]
val_id_list=[]
val_image_tensor = torch.empty((len(val_image_list),image_count,3, img_size, img_size))
for i in tqdm(range(len(val_image_list))):
    folder_name=os.path.basename(val_image_list[i])
    dst_label=label_data.loc[label_data['일련번호']==int(folder_name[:-1])]
    dst_label=dst_label.loc[dst_label['구분값']==int(folder_name[-1])].reset_index()
    label=int(dst_label.loc[0]['OTE 원인'])
    val_id_list.append(folder_name)
    val_label_list.append(label-1) 
    image_file_list = glob(val_image_list[i]+'/*.jpg')
    if len(image_file_list)>image_count:
        image_index = torch.randint(low=0, high=len(
            image_file_list)-image_count, size=(1,))
        count = 0
        for index in range(image_count):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            val_image_tensor[i,count] = image
            count += 1
    else:
        count = 0
        for index in range(len(image_file_list)):
            image = 1-tf(Image.open(image_file_list[index]).resize((img_size,img_size)))
            val_image_tensor[i,count] = image
            count += 1
        for j in range(image_count-count):
            image = 1-tf(Image.open(image_file_list[j]).resize((img_size,img_size)))
            val_image_tensor[i,count] = image
            count += 1
            
train_dataset = CustomDataset(train_id_list,train_image_tensor, F.one_hot(torch.tensor(train_label_list).to(torch.int64)))
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataset = CustomDataset(val_id_list,val_image_tensor, F.one_hot(torch.tensor(val_label_list).to(torch.int64)))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)


100%|██████████| 213/213 [00:30<00:00,  7.04it/s]


In [57]:
class FeatureExtractor(nn.Module):
    """Feature extoractor block"""
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        cnn1= timm.create_model('efficientnet_b2', pretrained=True)
        self.feature_ex = nn.Sequential(*list(cnn1.children())[:-1])

    def forward(self, inputs):
        features = self.feature_ex(inputs)
        
        return features
    
class AttentionMILModel(nn.Module):
    def __init__(self, num_classes, image_feature_dim,feature_extractor_scale1: FeatureExtractor):
        super(AttentionMILModel, self).__init__()
        self.num_classes = num_classes
        self.image_feature_dim = image_feature_dim

        # Remove the classification head of the CNN model
        self.feature_extractor = feature_extractor_scale1
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(image_feature_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        
        # Classification layer
        self.classification_layer = nn.Linear(image_feature_dim, num_classes)
        self.dropout=torch.nn.Dropout(0.2)
    def forward(self, inputs):
        batch_size, num_tiles, channels, height, width = inputs.size()
        
        # Flatten the inputs
        inputs = inputs.view(-1, channels, height, width)
        
        # Feature extraction using the pre-trained CNN
        features = self.feature_extractor(inputs)  # Shape: (batch_size * num_tiles, 2048, 1, 1)
        
        # Reshape features
        features = features.view(batch_size, num_tiles, -1)  # Shape: (batch_size, num_tiles, 2048)
        
        # Attention mechanism
        attention_weights = self.attention(features)  # Shape: (batch_size, num_tiles, 1)
        attention_weights = F.softmax(attention_weights, dim=1)  # Normalize attention weights
        
        # Apply attention weights to features
        attended_features = torch.sum(features * attention_weights, dim=1)  # Shape: (batch_size, 2048)
        attended_features=self.dropout(attended_features)
        attended_features=F.relu(attended_features)
        # Classification layer
        logits = self.classification_layer(attended_features)  # Shape: (batch_size, num_classes)
        
        return logits
Feature_Extractor=FeatureExtractor()
model = AttentionMILModel(3,1408,Feature_Extractor)
model = model.to(device)
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-4)
summary(model,(batch_size,image_count,3,img_size,img_size))


Layer (type:depth-idx)                                  Output Shape              Param #
AttentionMILModel                                       [4, 3]                    --
├─FeatureExtractor: 1-1                                 [200, 1408]               --
│    └─Sequential: 2-1                                  [200, 1408]               --
│    │    └─Conv2d: 3-1                                 [200, 32, 128, 128]       864
│    │    └─BatchNormAct2d: 3-2                         [200, 32, 128, 128]       64
│    │    └─Sequential: 3-3                             [200, 352, 8, 8]          7,201,634
│    │    └─Conv2d: 3-4                                 [200, 1408, 8, 8]         495,616
│    │    └─BatchNormAct2d: 3-5                         [200, 1408, 8, 8]         2,816
│    │    └─SelectAdaptivePool2d: 3-6                   [200, 1408]               --
├─Sequential: 1-2                                       [4, 50, 1]                --
│    └─Linear: 2-2                          

In [4]:
start = time.time()
d = datetime.datetime.now()
now_time = f"{d.year}-{d.month}-{d.day} {d.hour}:{d.minute}:{d.second}"
print(f'[deeplearning Start]')
print(f'deeplearning Start Time : {now_time}')
MIN_loss=5000
train_loss_list=[]
val_loss_list=[]
train_acc_list=[]
sig=nn.Sigmoid()
val_acc_list=[]
MIN_acc=0
check_val=1000
for epoch in range(300):
    train=tqdm(train_dataloader)
    count=0
    running_loss = 0.0
    acc_loss=0
    model.train()
    for x, y in train:
        y = y.to(device).float()
        count+=1
        x=x.to(device).float()
        optimizer.zero_grad()  # optimizer zero 로 초기화
        predict = model(x).to(device)
        cost = F.cross_entropy(predict.softmax(dim=1), y) # cost 구함
        acc=accuracy(predict.softmax(dim=1).argmax(dim=1),y.argmax(dim=1))
        cost.backward() # cost에 대한 backward 구함
        optimizer.step() 
        running_loss += cost.item()
        acc_loss+=acc
        train.set_description(f"epoch: {epoch+1}/{50} Step: {count+1} loss : {running_loss/count:.4f} accuracy: {acc_loss/count:.4f}")
    train_loss_list.append((running_loss/count))
    train_acc_list.append((acc_loss/count).cpu().detach().numpy())  
    val=tqdm(val_dataloader)
    path_list=[]
    model.eval()
    val_count=0
    val_running_loss = 0.0
    val_acc_loss=0
    with torch.no_grad():
        for x,y in val:
            y = y.to(device).float()
            val_count+=1
            x=x.to(device).float()
            predict = model(x).to(device)
            cost = F.cross_entropy(predict.softmax(dim=1), y) # cost 구함
            acc=accuracy(predict.softmax(dim=1).argmax(dim=1),y.argmax(dim=1))
            val_running_loss+=cost.item()
            val_acc_loss+=acc
            val.set_description(f"epoch: {epoch+1}/{50} Step: {count+1} loss : {val_running_loss/val_count:.4f} accuracy: {val_acc_loss/val_count:.4f}")
        val_loss_list.append((val_running_loss/val_count))
        val_acc_list.append((val_acc_loss/val_count).cpu().detach().numpy())  
        if check_val>val_running_loss/val_count:
            check_val=val_running_loss/val_count
            torch.save(model.state_dict(), '../../model/mteg/attention_eff50_check.pt')
torch.save(model.state_dict(), '../../model/mteg/attention_eff50_MIL.pt')
end = time.time()
d = datetime.datetime.now()
now_time = f"{d.year}-{d.month}-{d.day} {d.hour}:{d.minute}:{d.second}"
print(f'deeplearning Time : {now_time}s Time taken : {start-end}')
print(f'[deeplearning End]')


[deeplearning Start]
deeplearning Start Time : 2023-12-19 11:49:20


epoch: 1/50 Step: 70 loss : 1.0096 accuracy: 0.5362:  13%|█▎        | 69/542 [00:30<03:27,  2.28it/s]


KeyboardInterrupt: 

In [33]:
F.sigmoid(torch.tensor(1)).

tensor(0.7311)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
cm = confusion_matrix(total_y.cpu().argmax(axis=1),total_prob.cpu().argmax(axis=1))
classes = ['Oropharynx','Tonguebase','Epiglottis']

cm_display = ConfusionMatrixDisplay(cm,
                              display_labels=classes).plot()
f1 = f1_score(total_y.cpu().argmax(axis=1),total_prob.cpu().argmax(axis=1), average='macro')

print(f'total f1-score= {f1}') 

In [ ]:
def getIntersection_Method2(a, b):
    indices = torch.zeros_like(a, dtype=torch.uint8)

    for elem in b:
        indices = indices | (a == elem).type(torch.uint8)

    intersection = a[indices.type(torch.bool)]
    return intersection

def dfaa(index,name_list,label_clip,path_list):
    for i in range(len(index)):
        id=path_list[index[i]][0]
        temp_label=label_clip.loc[label_clip['wake']==int(id[-1])]
        temp_label=temp_label.loc[temp_label['Serial Number']==int(id[:-1])]
        temp_label=temp_label.reset_index()
        file_name=temp_label.loc[0]['File Name']
        name_list.append(file_name)
    return name_list
    
y_label=total_y.cpu().argmax(axis=1)
prob_label=total_prob.cpu().argmax(axis=1)

label_clip=pd.read_csv('../../data/label.csv',encoding='cp949') 
file_name_list=[]
index_1_1=getIntersection_Method2(torch.where(y_label==0)[0],torch.where(prob_label==0)[0])[:76]
index_1_2=getIntersection_Method2(torch.where(y_label==0)[0],torch.where(prob_label==1)[0])[:18]
index_1_3=getIntersection_Method2(torch.where(y_label==0)[0],torch.where(prob_label==2)[0])[:3]
index_2_1=getIntersection_Method2(torch.where(y_label==1)[0],torch.where(prob_label==0)[0])[:24]
index_2_2=getIntersection_Method2(torch.where(y_label==1)[0],torch.where(prob_label==1)[0])[:96]
index_2_3=getIntersection_Method2(torch.where(y_label==1)[0],torch.where(prob_label==2)[0])[:6]
index_3_1=getIntersection_Method2(torch.where(y_label==2)[0],torch.where(prob_label==0)[0])[:7]
index_3_2=getIntersection_Method2(torch.where(y_label==2)[0],torch.where(prob_label==1)[0])[:10]
index_3_3=getIntersection_Method2(torch.where(y_label==2)[0],torch.where(prob_label==2)[0])[:31]

file_name_list=dfaa(index_1_1,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_1_2,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_1_3,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_2_1,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_2_2,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_2_3,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_3_1,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_3_2,file_name_list,label_clip,path_list)
file_name_list=dfaa(index_3_3,file_name_list,label_clip,path_list)


In [ ]:
pd.DataFrame(file_name_list).to_csv('../../data/test.csv')

In [ ]:
pd.read_csv('../../data/test.csv',encoding='cp949') 



In [ ]:
list